In [1]:
import qml
import numpy as np

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qi

# Import data

In [2]:
paths=qi.wrapper_alch_data()

In [47]:
# paths

In [3]:
# load data into list, count number of atoms per molecule
alchemy_data, molecule_size = qi.load_alchemy_data(paths)
max_size = np.amax(molecule_size)

# Generate Representation

In [4]:
full_matrix = qi.generate_atomic_representations(alchemy_data, molecule_size)

In [5]:
full_matrix


array([[36.8581052 ,  2.90564209,  0.5       , ...,  0.        ,
         0.        ,  0.        ],
       [36.8581052 , 17.73882949, 73.51669472, ...,  0.        ,
         0.        ,  0.        ],
       [36.8581052 ,  2.90563574,  0.5       , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [53.3587074 , 19.54993299, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5       ,  2.92143402, 36.8581052 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5       ,  2.92677017, 36.8581052 , ...,  0.        ,
         0.        ,  0.        ]])

# Calculate distance between representations

In [19]:
dist_shape = int(len(full_matrix)*(len(full_matrix)+1)/2)
start=0
width = len(full_matrix)
dist = np.empty(dist_shape)
for idx in range(0, len(full_matrix)):
    dist[start:start+width] = np.linalg.norm(full_matrix[idx]-full_matrix[idx:], axis=1)
    start = start+width
    width -= 1

# Generate Label vector

In [8]:
energies = np.zeros(len(full_matrix))
start = 0
for idx, mol in enumerate(alchemy_data):
    length = len(alchemy_data[idx][:,6])
    energies[start:length+start] = alchemy_data[idx][:,6]
    start += length

# Difference between labels

In [9]:
diff_shape = int(len(full_matrix)*(len(full_matrix)+1)/2)
start=0
width = len(full_matrix)
en_diff = np.empty(diff_shape)
for idx in range(0, len(full_matrix)):
    en_diff[start:start+width] = np.abs(energies[idx]-energies[idx:])
    start = start+width
    width -= 1

# Plots

In [16]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})


In [44]:
# select random indices
indices = np.random.choice(len(dist), 100000)
indices = np.sort(indices)

fig, ax = plt.subplots(1,1)

ax.plot(np.take(dist, indices), np.take(en_diff, indices), 'o')
ax.set_xlabel(r'$d_{ij} $')
ax.set_ylabel('$|e_i - e_j|$')

Text(0, 0.5, '$|e_i - e_j|$')